In [32]:
import pandas as pd
import numpy as np


In [33]:
data = pd.read_csv("Assignment_2_Data_and_Template.csv")

In [34]:
data.head()

,Sex,Height,HandSpan
0,Female,68,21.5
1,Male,71,23.5
2,Male,73,22.5
3,Female,64,18.0
4,Male,68,23.5


In [35]:
def histClassifier(X,T, Bx ,By ,xmin,xmax):
    HF=np.zeros((Bx,Bx)).astype('int32')
    HM = np.zeros((By,By)).astype('int32')
    binindexHeight = (np.round((Bx-1)*(X['Height'] - xmin['Height'])/(xmax['Height'] - xmin['Height']))).astype('int32')
    binindexHandSpan = (np.round((By-1)*(X['HandSpan'] - xmin['HandSpan'])/(xmax['HandSpan'] - xmin['HandSpan']))).astype('int32')
    for (i,b) in enumerate(binindexHeight):
        if T[i] == 'Female':
            HF[b][binindexHandSpan[i]]+=1
        else:
            HM[b][binindexHandSpan[i]]+=1
    return [HF,HM]



In [36]:
data['Sex'].count()

167

In [37]:
import math
#count of male and female
countFemale = data[data['Sex'] == 'Female']['Height'].count()
print(countFemale)
countMale = data[data['Sex'] == 'Male']['Height'].count()
print(countMale)
N = data['Sex'].count()
BfemaleHist=math.ceil( math.log2(countFemale)+ 1)
BmaleHist=math.ceil( math.log2(countMale)+ 1)
print("B female - ", BfemaleHist)
print("B male - ", BmaleHist)

89
78
B female -  8
B male -  8


In [38]:
xmin = data.min()
xmax = data.max()
[HF,HM] = histClassifier(data,data['Sex'],BfemaleHist,BmaleHist,xmin,xmax)



In [39]:
xmin

Sex         Female
Height          57
HandSpan        16
dtype: object

In [40]:
xmax

Sex         Male
Height        78
HandSpan    25.5
dtype: object

In [41]:
HM

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  6,  1,  0,  0],
       [ 0,  1,  0,  2,  7,  8,  3,  0],
       [ 0,  0,  0,  5, 14,  7,  5,  0],
       [ 0,  0,  0,  1,  3,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1]], dtype=int32)

In [42]:
ht = 69
hs = 23.5
heightBin = (np.round((BfemaleHist-1)*(ht - xmin['Height'])/(xmax['Height'] - xmin['Height']))).astype('int32')
headSpanBin = (np.round((BfemaleHist-1)*(hs - xmin['HandSpan'])/(xmax['HandSpan'] - xmin['HandSpan']))).astype('int32')
print(heightBin)
print(headSpanBin)
print(HF[heightBin][headSpanBin])
print(HF[heightBin][headSpanBin] + HM[heightBin][headSpanBin])

4
6
0
3


In [43]:
HF[heightBin][headSpanBin]/(HF[heightBin][headSpanBin] + HM[heightBin][headSpanBin])

0.0

In [44]:
#PDF functions
meanFemale = data[['Height','HandSpan']][data['Sex'] == 'Female'].mean()
meanFemale

Height      65.280899
HandSpan    19.601124
dtype: float64

In [45]:
meanMale = data[['Height','HandSpan']][data['Sex'] == 'Male'].mean()
meanMale

Height      71.294872
HandSpan    22.301282
dtype: float64

In [46]:
CoFemale = data[['Height','HandSpan']][data['Sex'] == 'Female'].cov()
CoFemale

,Height,HandSpan
Height,7.727017,1.647408
HandSpan,1.647408,1.756703


In [47]:
CoMale = data[['Height','HandSpan']][data['Sex'] == 'Male'].cov()
CoMale

,Height,HandSpan
Height,7.041792,1.793124
HandSpan,1.793124,2.060648


In [48]:
type(CoFemale)

pandas.core.frame.DataFrame

In [49]:
a = CoFemale.as_matrix()
Femaledet = np.linalg.det(a)
Femaledet

10.86012331402638

In [50]:
m = CoMale.as_matrix()
Maledet = np.linalg.det(m)
Maledet

11.295359402502296

In [51]:
import math

In [52]:
import math
def pdf(mean, coMatrix, coDet, x):
    var = float(coDet)**0.5
    denom = (2*math.pi*var)
    xsubmean = np.matrix(x-mean)
    invcovariance = np.linalg.inv(coMatrix)
    meanInvProduct = xsubmean* invcovariance
    xsubMeanTranspose= np.transpose([np.array(x-mean)])
    num = math.exp(-1 *meanInvProduct *xsubMeanTranspose /2 )
    return num/denom

In [53]:
x = 69
y = 23.5
pdfFemale = pdf(meanFemale,a,Femaledet,[x,y])
pdfMale = pdf(meanMale,m,Maledet,[x,y])
ProbFemale = countFemale * pdfFemale / (countMale * pdfMale + countFemale * pdfFemale  )
print("Probabilty of females ",ProbFemale)

Probabilty of females  0.0567654529895


In [54]:
pdfFemale

0.0006378658525545122

In [55]:
BfemaleHist

8

In [56]:
#consturct histogram from baysian
#female
#number of bins for female histogram = BfemaleHist
#number of bins for male histogram = BmaleHist
#min - xmin
#max - xmax
HFplot = np.zeros((BfemaleHist,BfemaleHist))
HMplot = np.zeros((BfemaleHist,BfemaleHist))
binsizeHeight = (xmax['Height'] - xmin['Height'])/BfemaleHist
binsizeHandSpan =  (xmax['HandSpan'] - xmin['HandSpan'])/ BfemaleHist
xstart = xmin['Height']
ystart = xmin['HandSpan']
print("height bin sizee ", binsizeHeight , " handspan bin size ", binsizeHandSpan )
for i in range(BfemaleHist):
    print(" i = " , i)
    x = xstart + binsizeHeight/2 
    ydup = ystart
    for j  in range(BfemaleHist):
        y = ydup + binsizeHandSpan/2
        pdfFemale = pdf(meanFemale,a,Femaledet,[x,y])
        pdfMale = pdf(meanMale,m,Maledet,[x,y])
        HFplot[i][j] = pdfFemale * countFemale *  ( binsizeHeight * binsizeHandSpan)
        print("Centers" ,x, y )
        print(pdfMale)
        HMplot[i][j] = pdfMale * countMale *  ( binsizeHeight * binsizeHandSpan)
        ydup = ydup + binsizeHandSpan
    xstart = xstart + binsizeHeight
   
   

height bin sizee  2.625  handspan bin size  1.1875
 i =  0
Centers 58.3125 16.59375
4.979561315080014e-08
Centers 58.3125 17.78125
1.8987732992104192e-07
Centers 58.3125 18.96875
3.0057735633858056e-07
Centers 58.3125 20.15625
1.9753339025650695e-07
Centers 58.3125 21.34375
5.389219686171017e-08
Centers 58.3125 22.53125
6.103969716302525e-09
Centers 58.3125 23.71875
2.870119410474879e-10
Centers 58.3125 24.90625
5.602587837070055e-12
 i =  1
Centers 60.9375 16.59375
1.2340541931652497e-06
Centers 60.9375 17.78125
7.71838869332777e-06
Centers 60.9375 18.96875
2.0041036173044954e-05
Centers 60.9375 20.15625
2.1603037077294464e-05
Centers 60.9375 21.34375
9.667419466345253e-06
Centers 60.9375 22.53125
1.7960045292978973e-06
Centers 60.9375 23.71875
1.3851774250422462e-07
Centers 60.9375 24.90625
4.4351110163373576e-09
 i =  2
Centers 63.5625 16.59375
8.700320253765313e-06
Centers 63.5625 17.78125
8.925612627915903e-05
Centers 63.5625 18.96875
0.0003801385753349739
Centers 63.5625 20.15625

In [57]:
xmin

Sex         Female
Height          57
HandSpan        16
dtype: object

In [58]:
xmax

Sex         Male
Height        78
HandSpan    25.5
dtype: object

In [59]:
HFplot.sum()

88.697421619252168

In [64]:
np.round(HFplot)

array([[  0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  1.,   3.,   4.,   2.,   0.,   0.,   0.,   0.],
       [  1.,   5.,  11.,   8.,   2.,   0.,   0.,   0.],
       [  0.,   3.,  10.,  12.,   5.,   1.,   0.,   0.],
       [  0.,   1.,   3.,   6.,   4.,   1.,   0.,   0.],
       [  0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [61]:
HF

array([[ 1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  2,  2,  0,  0,  0,  0],
       [ 1,  5, 11,  8,  5,  0,  0,  0],
       [ 0,  1,  6, 20,  4,  0,  0,  0],
       [ 0,  0,  4,  8,  4,  2,  0,  0],
       [ 0,  0,  1,  1,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [62]:
HM

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  6,  1,  0,  0],
       [ 0,  1,  0,  2,  7,  8,  3,  0],
       [ 0,  0,  0,  5, 14,  7,  5,  0],
       [ 0,  0,  0,  1,  3,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1]], dtype=int32)